# Video compression

The following use `ffmpeg`:
* General compression for the web 1: https://dev.to/benjaminblack/use-ffmpeg-to-compress-and-convert-videos-458l  
* General compression for the web 2: https://trac.ffmpeg.org/wiki/Encode/YouTube   
* Frame rate change: https://trac.ffmpeg.org/wiki/ChangingFrameRate  
* Scaling: https://trac.ffmpeg.org/wiki/Scaling  

In [ ]:
from IPython.display import HTML
from fastai.core import *
from fastai.vision import *
from kgl_deepfake.data import *

In [ ]:
SOURCE = Path('../data/dfdc_train_part_0/')

In [ ]:
fnames = get_files(SOURCE, extensions=['.mp4'])

In [ ]:
fn = fnames[19]
fn

PosixPath('../data/dfdc_train_part_0/goxbkycgkg.mp4')

In [ ]:
def get_file_size(fpath):
    assert fpath.is_file()
    p = subprocess.run(f"du -hs {fpath}".split(), stdout=subprocess.PIPE)
    return p.stdout.decode().split('\t')[0]

In [ ]:
Path.file_size = get_file_size

Original file size

In [ ]:
fn.file_size()

'8.6M'

In [ ]:
def runnit(f):
    def _func(*args):
        _, fpath = args
        if fpath.exists(): os.remove(fpath)
        command = f(*args)
        p = subprocess.run(command.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return p
    return _func

In [ ]:
def show_vid_info(f):
    def _f(*args):
        p = f(*args)
        _, fpath = args
        print(fpath.file_size())
        display(HTML(html_vid(fpath)))
    return _f

In [ ]:
@show_vid_info
@runnit
def general_compress_1(fpath_from, fpath_to):
    "General compression for the web"
    return f"ffmpeg -i {fpath_from} -c:v libx264 -b:v 1.5M -c:a aac -b:a 128k {fpath_to}"

@show_vid_info
@runnit
def general_compress_2(fpath_from, fpath_to):
    "General compression for the web"
    return f"ffmpeg -i {fpath_from} -c:v libx264 -preset slow -crf 18 -c:a aac -b:a 192k -pix_fmt yuv420p {fpath_to}"

@show_vid_info
@runnit
def change_fps(fpath_from, fpath_to):
    "Change FPS (frames per second)"
    return f"ffmpeg -i {fpath_from} -filter:v fps=fps=15 {fpath_to}"

def scaling(fpath_from, fpath_to):
    "Scaleling."
    return f'''ffmpeg -i {fpath_from} -vf "scale=iw/4:ih/4" {fpath_to}'''

In [ ]:
fpath_from = fn

### Compression results

General compression 1

In [ ]:
%%time
fpath_cmp1 = Path(f'cmp1_{fn.name}')
general_compress_1(fpath_from, fpath_cmp1)

2.0M


CPU times: user 6.19 ms, sys: 12.7 ms, total: 18.9 ms
Wall time: 11.8 s


General compression 2

In [ ]:
%%time
fpath_cmp2 = Path(f'cmp2_{fn.name}')
general_compress_1(fpath_from, fpath_cmp2)

2.0M


CPU times: user 5.61 ms, sys: 10.3 ms, total: 15.9 ms
Wall time: 12.3 s


FPS reduced to 15

In [ ]:
%%time
fpath_fps15 = Path(f'fps15_{fn.name}')
change_fps(fpath_from, fpath_fps15)

6.0M


CPU times: user 5.92 ms, sys: 11.4 ms, total: 17.3 ms
Wall time: 12.2 s


lengths ---> lengths / 4

In [ ]:
%%time
fpath_smll = Path(f'smll_{fn.name}')
change_fps(fpath_from, fpath_smll)

6.0M


CPU times: user 7.34 ms, sys: 11.6 ms, total: 18.9 ms
Wall time: 13 s


# -fin